<div style="width:100%; background-color: #000041"><a target="_blank" href="http://university.yugabyte.com"><img src="assets/YBU_Logo.png" /></a></div>

> **YugabyteDB YSQL Development**
>
> Enroll for free  [Yugabyte University](https://university.yugabyte.com/).
>

# About these notebook Labs
The labs for this course consists of several Jupyter notebooks that run in Gitpod. [Gitpod](https://www.gitpod.io/) facilitates runtime environments using Ubuntu and VS Code in a Chromium (browser) environment. 

A Gitpod instance belongs to your Gitpod workspace. The instance automatically runs a three node, YugabyteDB database cluster. YugabyteDB is the world's #1 open source, distributed SQL database. 

> Although a three node cluster is up and running, Gitpod does not support visiting loopback addresses over a web ui, even if exposed on a different port.
> 127.0.0.1 is the only web user interface that is exposed. To see all available ports in Gitpod, in the Terminal tab, you can run `gp ports list`.
> 

## 🛠️ Requirements
Here are the requirements for this lab:
- ✅ Launch using this Gitpod workspace, which you are already doing
- ✅ A running, 3 node YugabyteDB cluster, which is already running
- ☑️ A Python 3.8.13 or higher kernel for the notebook, *which you need to select right now!!!*

### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.8.13** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!

## About Jupyter notebooks

A Juypter notebook is a runtime environment for executing code. A notebook consists of one or more cells. There are three types of cells in a notebook: Markdown, Code, and Output. This is a Markdown cell.

A cell has two modes: read-mode and edit-mode. By default, the cells in a notebook are in read-mode. You can run a read-mode cell that is code. To execute a code cell, select the play icon in the cell's left gutter. 

The execution of a code cell often results in Output. An Output cell shows the `stdout` of the previous executed code cell. You can delete an Output cell. You can also collapse the cell. In addition, you can also select `Clear Outputs of All Cells`.

To modify a cell, select `Enter` or `Return`. In edit-mode, you can modify the contents of a cell. Certain labs contain challenges or experiments that require you to do just that: modify a code cell and re-run it! To switch from edit-mode to read-mode, use the `Shift+Enter` or `Shift+Return` keystroke.

#### Notebook keyboard shortcuts
The Jupyter extension for Gitpod supports the following keyboard shortcuts:
| Keystroke | Description |
|--|--|
| ESC | Change the cell mode |
| A | Add a cell above |
| B | Add a cell below |
| J or down arrow key |  Change a cell to below | 
| K or up arrow key | Change a cell to above | 
| Ctrl+Enter | Run the currently selected cell |
| Shift+Enter | Run the currently selected cell and insert a new cell immediately below (focus moves to new cell) |
| Alt+Enter | Run the currently selected cell and insert a new cell immediately below (focus remains on current cell) |
| dd | Delete a selected cell |
| z | Undo the last change | 
| M | switch the cell type to Markdown | 
| Y | switch the cell type to code |
| L | Enable/Disable line numbers |
```


## ⛑️ Getting help
The best way to get help from the Yugabyte University team is to post your question on YugabyteDB Community Slack in the #training or #yb-university channels. To sign up, visit [https://communityinviter.com/apps/yugabyte-db/register](https://communityinviter.com/apps/yugabyte-db/register).

---
# 👣 Setup steps
Here are the steps to setup this lab:
- Install missing dependencies and restart the notebook
- Create the notebook variables
- Create the `db_ybu` database

### Install missing dependencies and restart the notebook
Run the following cell to ensure that the notebook dependencies are available to the notebook. 

In [ ]:
!pip install --upgrade pip
!pip install ipython-sql~=0.5 --upgrade
!pip install psycopg2-binary
!pip install SQLAlchemy --upgrade 
!pip install sqlparse
!pip install html2text
!pip install jq

### Create the notebook variables 

> IMPORTANT!
> 
> Do NOT skip running this cell. 
> 

The following Python cell creates and stores variables that all the notebooks in this lab will use. You can view these variables in the Jupyter tab.

- To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.
- Verify the accuracy of the output values

In [ ]:
# Env variables for Notebook
import os

# read env_vars.env
env_vars = !cat env_vars.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

### Begin: Gitpod or local
#
### Uncomment Gitpod specific
#
MY_YB_PATH=os.environ.get('MY_YB_PATH_GITPOD')
MY_YB_PATH_DATA=os.environ.get('MY_YB_PATH_DATA_GITPOD')
MY_GITPOD_WORKSPACE_URL=os.environ.get('GITPOD_WORKSPACE_URL')
#
### Comment out Local 
#
# MY_YB_PATH=os.environ.get('MY_YB_PATH_LOCAL')
# MY_YB_PATH_DATA=os.environ.get('MY_YB_PATH_DATA_LOCAL')
# MY_GITPOD_WORKSPACE_URL=os.environ.get('MY_GITPOD_WORKSPACE_URL_LOCAL')
##
### End: Gitpod or local

# env_vars defines the following
MY_DB_NAME=os.environ.get('MY_DB_NAME')
MY_HOST_IPv4_01=os.environ.get('MY_HOST_IPv4_01')
MY_HOST_IPv4_02=os.environ.get('MY_HOST_IPv4_02')
MY_HOST_IPv4_03=os.environ.get('MY_HOST_IPv4_03')
MY_TSERVER_WEBSERVER_PORT=os.environ.get('MY_TSERVER_WEBSERVER_PORT')

# Gitpod URLS
MY_YB_MASTER_HOST_GITPOD_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
MY_YB_TSERVER_HOST_GITPOD_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://'+MY_TSERVER_WEBSERVER_PORT+'-')

# Current directory of project and related child folders
MY_NOTEBOOK_DIR=os.getcwd()
MY_NOTEBOOK_DATA_FOLDER=MY_NOTEBOOK_DIR +'/data'
MY_NOTEBOOK_UTILS_FOLDER=MY_NOTEBOOK_DIR + '/utils'

# Data files
MY_DATA_DDL_FILE_0=os.environ.get("MY_DATA_DDL_FILE_0")
MY_DATA_DML_FILE_0=os.environ.get("MY_DATA_DML_FILE_0")
MY_DATA_DDL_FILE_1=os.environ.get("MY_DATA_DDL_FILE_1")
MY_DATA_DML_FILE_1=os.environ.get("MY_DATA_DML_FILE_1")
MY_DATA_DDL_FILE_2=os.environ.get("MY_DATA_DDL_FILE_2")
MY_DATA_DML_FILE_2=os.environ.get("MY_DATA_DML_FILE_2")
MY_DATA_DDL_FILE_3=os.environ.get("MY_DATA_DDL_FILE_3")
MY_DATA_DML_FILE_3=os.environ.get("MY_DATA_DML_FILE_3")

MY_JEOPARDY_DATA_FILE=os.environ.get("MY_JEOPARDY_DATA_FILE")
MY_GIN_EXAMPLES=os.environ.get("MY_GIN_EXAMPLES")
MY_GITHUB_DATA_FILE=os.environ.get("MY_GITHUB_DATA_FILE")

# Util files
MY_UTIL_FUNCTIONS_FILE=os.environ.get("MY_UTIL_FUNCTIONS_FILE")
MY_UTIL_YBTSERVER_METRICS_FILE=os.environ.get("MY_UTIL_YBTSERVER_METRICS_FILE")


# Store the note book values for other notebooks to use

%store MY_DB_NAME
%store MY_YB_PATH
%store MY_YB_PATH_DATA
%store MY_GITPOD_WORKSPACE_URL
%store MY_HOST_IPv4_01
%store MY_HOST_IPv4_02
%store MY_HOST_IPv4_03
%store MY_TSERVER_WEBSERVER_PORT
%store MY_YB_MASTER_HOST_GITPOD_URL
%store MY_YB_TSERVER_HOST_GITPOD_URL
%store MY_NOTEBOOK_DIR
%store MY_NOTEBOOK_DATA_FOLDER
%store MY_NOTEBOOK_UTILS_FOLDER
%store MY_DATA_DDL_FILE_0
%store MY_DATA_DML_FILE_0
%store MY_DATA_DDL_FILE_1
%store MY_DATA_DML_FILE_1
%store MY_DATA_DDL_FILE_2
%store MY_DATA_DML_FILE_2
%store MY_DATA_DDL_FILE_3
%store MY_DATA_DML_FILE_3


%store MY_JEOPARDY_DATA_FILE
%store MY_GIN_EXAMPLES
%store MY_GITHUB_DATA_FILE

%store MY_UTIL_FUNCTIONS_FILE
%store MY_UTIL_YBTSERVER_METRICS_FILE

---
# Connect to YugabyteDB with `ysqlsh`, YSQL shell, and create the database
The following commands are YSQL shell commands. Using `ysqlsh`, you can execute YSQL statements.

Run the following cell to connect to the local host using `ysqlsh`, create the `db_ybu` database, and then list the databases.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# terminate connections, drop, and create
./bin/ysqlsh -d yugabyte -c "select pg_terminate_backend(pid) from pg_stat_activity where datname = '"${DB_NAME}"';"  
./bin/ysqlsh -d yugabyte -c "drop database if exists "${DB_NAME}";"  
./bin/ysqlsh -d yugabyte -c "create database "${DB_NAME}";" 

# list dbs
./bin/ysqlsh -d yugabyte -c "\l"

### Create Utils
For illustration purposes, this lab requires specific user defined functions and objects for the following:
- Functions for converting hash range strings to integers
- YB-TServer metrics

#### Util Functions
Run the following cell to create the utility functions.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_UTILS_FOLDER" "$MY_UTIL_FUNCTIONS_FILE"  # util functions
YB_PATH=${1}
DB_NAME=${2}
UTILS_FOLDER=${3}
UTIL_FUNCTIONS_FILE=${4}


#ls $UTIL_FOLDER
UTIL_FUNCTIONS_FILE_PATH=${UTILS_FOLDER}/${UTIL_FUNCTIONS_FILE}

cd $YB_PATH

# Functions file
./bin/ysqlsh -q -d ${DB_NAME} -f ${UTIL_FUNCTIONS_FILE_PATH} 
sleep 1;

# Describe functions
./bin/ysqlsh -d ${DB_NAME} -c "\df public.fn_*"

#### Util Metrics
Run the following cell to create the utility functions for the metrics report.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_UTILS_FOLDER" "$MY_UTIL_YBTSERVER_METRICS_FILE"  # utl metrics
YB_PATH=${1}
DB_NAME=${2}
UTILS_FOLDER=${3}
UTIL_YBTSERVER_METRICS_FILE=${4}

#ls $UTIL_FOLDER

UTIL_YBTSERVER_METRICS_FILE=${UTILS_FOLDER}/${UTIL_YBTSERVER_METRICS_FILE}

cd $YB_PATH

# Metrics file
./bin/ysqlsh -d ${DB_NAME} -f ${UTIL_YBTSERVER_METRICS_FILE} >&/dev/null

# Describe relations
./bin/ysqlsh -d ${DB_NAME} -c "\df public.fn_yb*"

##### Create tables, load data, and review relations
Run the following cell to execute the DDL and DML scripts using `ysqlsh`.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_DATA_FOLDER" "$MY_DATA_DDL_FILE_0" "$MY_DATA_DML_FILE_0"   # World Cities
YB_PATH=${1}
DB_NAME=${2}
DATA_FOLDER=${3}
DATA_DDL_FILE=${4}
DATA_DML_FILE=${5}

#ls $DATA_FOLDER

WORLD_DDL_PATH=${DATA_FOLDER}/${DATA_DDL_FILE}
WORLD_DML_PATH=${DATA_FOLDER}/${DATA_DML_FILE}

cd ${YB_PATH}

# DDL file
./bin/ysqlsh -d ${DB_NAME} -f ${WORLD_DDL_PATH} >&/dev/null
sleep 1;

# DML file
./bin/ysqlsh -d ${DB_NAME} -f ${WORLD_DML_PATH} >&/dev/null
sleep 1;

# Describe relations
./bin/ysqlsh -d ${DB_NAME} -c "\d public.*"

##### View DDL for tbl_countries
Run the following cell using `ysqlsh` to view a table definition.

> Note
> 
> SQL magic does not support PostgreSQL `psql` commands. In order to execute `psql` commands, the notebook uses bash and `ysqlsh`.



In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # \d tbl_countriees

YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH


./bin/ysqlsh -d ${DB_NAME} -c "\d tbl_countries"
#./bin/ysqlsh -d ${DBNAME} -c "\d tbl_countries"
#./bin/ysqlsh -d ${DBNAME} -c "\d tbl_states"


## Connect to YugabyteDB using the PostgreSQL Driver for Python
The following cells requires:
- Python 3.8+ and psycopg2

In [ ]:
# Connect to db_ybu
# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# env_var.env
db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME

connection_str='postgresql://yugabyte@'+db_host+':5433/'+db_name
print(connection_str)
engine = create_engine(connection_str)

#### Load SQL magic extension
>IMPORTANT!
>
> To use SQL magic, you must run the following cell that loads the notebook extension.

In [ ]:
%reload_ext sql
# creates connection for sql magic
%sql {connection_str}

Optional.

In [ ]:
 %config SqlMagic

#### Show table row counts
Run the cell below to view the row counts for the tables.

In [ ]:
%%sql 

select '' _
    , v1.name
    , v1.counts
from (
    select 'tbl_cities' as name, count(*) as counts from tbl_cities
    union 
    select 'tbl_cities_name_alt_null' as name, count(*) 
    from tbl_cities
    where 1=1
    and city_name_alt IS NULL
    union
    select 'tbl_states' as name,count(*) from tbl_states
    union 
    select 'tbl_countries' as name, count(*) from tbl_countries
    union
    select 'tbl_country_name_alt_null' as name, count(*) 
    from tbl_countries 
    where 1=1
    and country_name_alt IS NULL
    ) v1
order by v1.counts desc;

---
## Add Node and view the data distribution

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl create
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH
echo $YB_PATH

# Add node
./bin/yb-ctl add_node \
--tserver_flags "yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=false,webserver_port="${TSERVER_WEBSERVER_PORT}  \
--placement_info "cloud1.region1.zone1" 


---
## Remove Node and view the data distribution

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl create
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH
echo $YB_PATH

# Add node
#./bin/yb-ctl remove_node 4
./bin/yb-ctl status


---
# 🌟 Well done! 
In this notebook, you completed the following:
- Setup steps
- Connect to YugabyteDB using ysqlsh
  - Created the `db_ybu` database with `ysqlsh`
  - Created utils
  - Created tables and loaded data using DDL and DML scripts
  - Connected to the database using a PostgreSQL driver for Python
  - Executed queries with SQL Magic


## 😊 Next up!
Continue your learning by opening the next notebook, `02_Demystifying_table_sharding_tablets_and_data_distribution`. 

Or, if using GitPod, run the following cell:

In [ ]:
%%bash
gp open '02_Demystifying_table_sharding_tablets_and_data_distribution'